In [14]:
import pandas as pd
import os
wdir = '/Users/candicewithrow/code/niffler/niffler'


In [2]:
pwd

'/Users/candicewithrow/code/niffler/niffler'

In [3]:
import json
with open(os.path.join(wdir,'output/full_02_03_2019.json'), 'r') as f:
                nested_data = json.load(f)
           

In [4]:
for rec in nested_data:
    rec['post_id'] = rec['id']
    rec.pop('id', None)
    for c in rec['comments']:
        c['cid'] = c['id']
        c.pop('id', None)
        if 'replies' in c.keys():
            for r in c['replies']:
                if 'id' in r.keys():
                    r['rid'] = r['id']
                    r.pop('id', None)
    

In [5]:
nested_data

[{'comments': [{'cid': '10155966481665857',
    'created_time': '2019-01-24T20:05:45+0000',
    'message': 'Not ocular but yes migraines are my primary reaction to sulphites. Best wishes',
    'replies': [{'created_time': '2019-01-25T10:57:54+0000',
      'message': 'Clare Ross same',
      'rid': '10155967531485857'}]},
   {'cid': '10155966792785857',
    'created_time': '2019-01-24T22:58:51+0000',
    'message': 'I  have  ocular  migraines  occasionally,  but  never  thought  to  link  them  to  sulfites.  I  do  remember  once  when  it  happened  i I was  getting  over  a  cold.  I  do  get  headaches  that  can  progress  to  migraines  from  certain  sulfites.'},
   {'cid': '10155966795795857',
    'created_time': '2019-01-24T23:02:44+0000',
    'message': 'My ocular migraines decreased significantly after cutting out sulfites, and migraines are a sure thing if I happen to eat them. I have meds and migraine contacts to cope'},
   {'cid': '10155966868725857',
    'created_time': '

In [17]:
posts=pd.read_json(os.path.join(wdir,'output/2019_02_28/posts2019_02_28.json'), 
                   orient='records')
comments=pd.read_json(os.path.join(wdir,'output/2019_02_28/comments2019_02_28.json'), 
                      orient='records')
replies=pd.read_json(os.path.join(wdir,'output/2019_02_28/replies2019_02_28.json'), orient='records')

In [16]:
posts.head(5)

,post,post_id,updated_time
0,"So, do I have this correct? The baking powder...",6.456587e+27,2019-02-26 14:17:57
1,Is there anything you would add to \nthis docu...,6.456587e+27,2019-02-26 13:43:02
2,Can someone explain this to me please? MAgnesi...,6.456587e+27,2019-02-26 08:44:45
3,"Couldn't see much about calamine lotion,is it ...",6.456587e+27,2019-02-26 02:56:59
4,"I had an allergic reaction at the dentist, pro...",6.456587e+27,2019-02-25 21:26:08


In [18]:
comments.head()

,comm_id,comment,created_time,post_id
0,10155668654010856,"I thought I suggested Hain, as I contacted the...",2018-08-25 14:59:05,6.456587e+27
1,10155668757310856,Most baking powder the issue is the corn starc...,2018-08-25 15:50:54,6.456587e+27
2,10155668870210856,"So, Janice, do you mean whole milk straight fr...",2018-08-25 17:07:34,6.456587e+27
3,10156022084220856,I just bought the Bakewell product. It should ...,2019-02-21 20:26:50,6.456587e+27
4,10156026653210856,Following,2019-02-24 05:05:47,6.456587e+27


In [19]:
replies.head()


,comm_id,created_time,post_id,rep_id,reply
0,10155668654010856,2018-08-25 15:14:12,6.456587e+27,10155668679545856,"Yes, Liz, Hain is also on our shopping list. ..."
1,10155668654010856,2018-08-25 15:25:53,6.456587e+27,10155668708010856,Kathy O'Connor Van Corbach This is the email I...
2,10155668654010856,2018-08-25 16:16:15,6.456587e+27,10155668793820856,"Yes, I found that response. I was just thinki..."
3,10155668654010856,2018-08-25 16:21:39,6.456587e+27,10155668800955856,Kathy O'Connor Van Corbach I have been that si...
4,10155668654010856,2018-08-25 17:01:33,6.456587e+27,10155668860965856,"I'm definitely going to try it then, Liz! Tha..."


In [20]:
full_df = pd.merge(posts, comments, on='post_id')


In [21]:
full_df

,post,post_id,updated_time,comm_id,comment,created_time
0,"So, do I have this correct? The baking powder...",6.456587e+27,2019-02-26 14:17:57,10155668654010856,"I thought I suggested Hain, as I contacted the...",2018-08-25 14:59:05
1,"So, do I have this correct? The baking powder...",6.456587e+27,2019-02-26 14:17:57,10155668757310856,Most baking powder the issue is the corn starc...,2018-08-25 15:50:54
2,"So, do I have this correct? The baking powder...",6.456587e+27,2019-02-26 14:17:57,10155668870210856,"So, Janice, do you mean whole milk straight fr...",2018-08-25 17:07:34
3,"So, do I have this correct? The baking powder...",6.456587e+27,2019-02-26 14:17:57,10156022084220856,I just bought the Bakewell product. It should ...,2019-02-21 20:26:50
4,"So, do I have this correct? The baking powder...",6.456587e+27,2019-02-26 14:17:57,10156026653210856,Following,2019-02-24 05:05:47
5,"So, do I have this correct? The baking powder...",6.456587e+27,2019-02-26 14:17:57,10156030892700856,We've been eating these pancakes though we hav...,2019-02-26 14:17:51
6,"So, do I have this correct? The baking powder...",6.456587e+27,2019-02-26 14:17:57,10155537772190856,Sulfur... sulfate.. and sulfite are different ...,2018-06-24 14:07:48
7,"So, do I have this correct? The baking powder...",6.456587e+27,2019-02-26 14:17:57,10155539032200856,I have anaphylactic reactions to sulFITES but ...,2018-06-25 03:38:43
8,"So, do I have this correct? The baking powder...",6.456587e+27,2019-02-26 14:17:57,10155539486255856,This allergy seems impossible to master 🤦,2018-06-25 12:03:31
9,"So, do I have this correct? The baking powder...",6.456587e+27,2019-02-26 14:17:57,10155539487310856,So is it possible to be allergic to sulFATES?,2018-06-25 12:04:37


In [12]:
from pandas.io.json import json_normalize
posts = json_normalize(nested_data)[['post_id','message','updated_time']]
comments = json_normalize(nested_data, record_path='comments', meta=['post_id'])
comments.head(3)

,cid,created_time,message,replies,post_id
0,10155966481665857,2019-01-24T20:05:45+0000,Not ocular but yes migraines are my primary re...,"[{'message': 'Clare Ross same', 'created_time'...",64565870856_10155966469605857
1,10155966792785857,2019-01-24T22:58:51+0000,"I have ocular migraines occasionally, but...",NaN,64565870856_10155966469605857
2,10155966795795857,2019-01-24T23:02:44+0000,My ocular migraines decreased significantly af...,NaN,64565870856_10155966469605857


In [8]:
full_df = pd.merge(posts, comments, on='post_id')
full_df[['message_x','message_y', 'created_time', 'updated_time']].head(5)

,message_x,message_y,created_time,updated_time
0,Do any of you get ocular migraines? Any relati...,Not ocular but yes migraines are my primary re...,2019-01-24T20:05:45+0000,2019-02-02T19:04:06+0000
1,Do any of you get ocular migraines? Any relati...,"I have ocular migraines occasionally, but...",2019-01-24T22:58:51+0000,2019-02-02T19:04:06+0000
2,Do any of you get ocular migraines? Any relati...,My ocular migraines decreased significantly af...,2019-01-24T23:02:44+0000,2019-02-02T19:04:06+0000
3,Do any of you get ocular migraines? Any relati...,My daughter’s primary reaction to sulfites is ...,2019-01-24T23:56:29+0000,2019-02-02T19:04:06+0000
4,Do any of you get ocular migraines? Any relati...,I used to get ocular migraines but haven’t had...,2019-01-25T00:44:44+0000,2019-02-02T19:04:06+0000


In [13]:
full_df['post_updated']=pd.to_datetime(full_df['updated_time'])
full_df['message_created']=pd.to_datetime(full_df['created_time'])
full_df[['message_x','message_y', 'post_updated', 'message_created']].head(2)

,message_x,message_y,post_updated,message_created
0,Do any of you get ocular migraines? Any relati...,Not ocular but yes migraines are my primary re...,2019-02-02 19:04:06,2019-01-24 20:05:45
1,Do any of you get ocular migraines? Any relati...,"I have ocular migraines occasionally, but...",2019-02-02 19:04:06,2019-01-24 22:58:51


In [10]:
full_df['updated_time'].apply(lambda x:pd.to_datetime(x).date().strftime('%m-%d-%Y')).head()


0    02-02-2019
1    02-02-2019
2    02-02-2019
3    02-02-2019
4    02-02-2019
Name: updated_time, dtype: object